# spicy snow counter example

This notebook shows how spicy snow / lievens algorithm might not actually be measuring changes in snow depth as well as we think it does. I hypothesize we are basically accumulating a (elevation dependent) backscatter ratio variability. This elevation dependent backscatter ratio variability is complicated, but likely includes some combination of radar processing affects, radar artifacts, elevation dependent temperature, soil moisture, as well as maybe a little true volume scattering from snow and wet snow effects (ask me about this one, I have some theories), etc. To start testing this hypothesis, I ran the spicy snow algorithm retrieving snow depth over an area that don't actually regularly accumulate snow (mountains just north of phenoix). My hypothesis is that if I get similar snow depth rasters for two different mountain ranges (one with true snow, and one without true snow), the physics and physical interpretation of this snow depth method should be in question. I set fake ims data (0 before the accumulation season, 4 in the accumulation season, and 0 afterwards to trick the algorithm into thinking there is snow present during the accumulation season), ran the algorithm, and found a believeable "snow depth". The "snow depth" raster time series is shown below in the facetgrid plot, and below that is the mean "snow depth" in each elevation bin over time.

In [ ]:
!pip install -q pystac-client
!pip install -q planetary-computer
!pip install -q odc-stac
!pip install -q xarray-spatial


In [ ]:
import sys
sys.path.append('../../../spicy-snow/')
import os
from spicy_snow.retrieval import retrieve_snow_depth, retrieval_from_parameters
from spicy_snow.IO.user_dates import get_input_dates
import geopandas as gpd
from pathlib import Path
from shapely import geometry
import contextily as ctx 
import matplotlib.pyplot as plt
import xarray as xr
import rioxarray as rxr
import pystac_client
import planetary_computer
import odc.stac
import numpy as np
import pandas as pd
import xrspatial.multispectral as ms

## choose mountainous area with usually no snow during accumulation season

In [ ]:
# geojson of aoi
aoi = 'north_of_pheonix'
area_gdf = gpd.read_file(f'data/{aoi}.geojson')
area = list(area_gdf.geometry)[0].envelope # convert feature to box for search

In [ ]:
# Plot aoi
fig, ax = plt.subplots(figsize=(12,12))
area_gdf.boundary.plot(ax=ax, color='r')

ctx.add_basemap(ax=ax, crs=area_gdf.crs, source=ctx.providers.OpenTopoMap)
ax.set_title('AOI location')

## retrieve snow depth per usual

In [ ]:
# this will generate a tuple of dates from the previous August 1st to this date
dates = get_input_dates('2021-07-31')

# define output directory and file name
out_nc = Path(f'data/{aoi}_sd_{dates[0]}_{dates[1]}.nc').expanduser()

if os.path.isfile(out_nc):
    spicy_ds = xr.open_mfdataset(out_nc)
    print(f'Opening dataset at {out_nc}')
else:
    spicy_ds = retrieve_snow_depth(area = area, dates = dates, 
                                work_dir = Path('/tmp/er_test/').expanduser(), 
                                job_name = f'sd_{dates[0]}_{dates[1]}',
                                existing_job_name = f'{aoi}_sd_{dates[0]}_{dates[1]}',
                                debug=False,
                                outfp=out_nc)

## add a dem layer and visualize

In [ ]:
# bring in cop 30 m dem
catalog = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1",modifier=planetary_computer.sign_inplace)
search = catalog.search(collections="cop-dem-glo-30", bbox=spicy_ds.rio.bounds())
items = list(search.get_items())

In [ ]:
dem_raster_all = odc.stac.load(items, bbox=spicy_ds.rio.bounds()).squeeze()
dem_raster = dem_raster_all['data']
dem_raster = dem_raster.rio.set_nodata(np.NaN)

dem = dem_raster.rio.reproject_match(spicy_ds)
spicy_ds['dem'] = dem

In [ ]:
spicy_ds['dem'].plot()

## let's check the included ims data to check if there is truly no snow in our aoi

In [ ]:
spicy_ds['ims'].plot(col='time',col_wrap=10)

## let's confirm with optical imagery

In [ ]:
search = catalog.search(
    bbox=spicy_ds.rio.bounds(),
    datetime=f'{dates[0]}/{dates[1]}',
    collections="sentinel-2-l2a")

items = list(search.get_items())

In [ ]:
data = (odc.stac.load(
        items,
        bands=["B04", "B03", "B02"],  # red, green, blue
        like=dem_raster_all)
    .where(lambda x: x > 0, other=np.nan)  # sentinel-2 uses 0 as nodata
    .assign_coords(band=lambda x: x.common_name.rename("band"))  # use common names
)

In [ ]:
data

In [ ]:
monthly = data.groupby("time.week").median()#.compute()

In [ ]:
images = [ms.true_color(*x) for x in monthly]
images = xr.concat(images, dim="time")

g = images.plot.imshow(x="x", y="y", rgb="band", col="time", col_wrap=5, figsize=(6, 8))
for ax in g.axes.flat:
    ax.set_axis_off()

plt.tight_layout()

## now we will replace the true ims data with fake ims data based on a fake accumulation season and visualize the new ims data

In [ ]:
fake_accumulation_season_start = '2020-10-15'
fake_accumulation_season_end = '2021-07-01'
spicy_ds['ims'] = xr.where((spicy_ds['ims'].time<pd.to_datetime(fake_accumulation_season_start)) | (spicy_ds['ims'].time>pd.to_datetime(fake_accumulation_season_end)),xr.full_like(spicy_ds['ims'].isel(time=0),0),xr.full_like(spicy_ds['ims'].isel(time=0),4))

In [ ]:
spicy_ds['ims'].plot(col='time',col_wrap=10)

## now we will rerun snow index and snow depth calculations with the fake ims data

In [ ]:
spicy_ds = retrieval_from_parameters(spicy_ds, A = 2.5, B = 0.2, C = 0.55, wet_snow_thresh = -2, freezing_snow_thresh = 1)

## "snow depth" raster time series

In [ ]:
spicy_ds['snow_depth'].plot(col='time',col_wrap=10,cmap='Blues')

## "snow depth" time series binned by elevation

In [ ]:
f,ax=plt.subplots(figsize=(24,7))
spicy_ds.groupby_bins(group='dem',bins=30).mean()['snow_depth'].plot(ax=ax,cmap='Blues')